In [1]:
import numpy as np
import json

item_matrix = np.load('item_matrix.npy', allow_pickle=True )
item_id = np.load('itemid.npy', allow_pickle=True)
wts = np.load('weights.npy', allow_pickle=True)

f = open('anime_id2name.json')

id2name = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
def topn_recommendations(scores, topn=5): 
    recommendations = np.apply_along_axis(topidx, 0, scores, topn) 
    return recommendations 
 
def topidx(a, topn): 
    parted = np.argpartition(a, -topn)[-topn:] 
    return parted[np.argsort(-a[parted])]

def downvote_seen_items(scores, seen_ids): 
    min_score = min(scores) - 1
    for id_ in seen_ids:
        pos = np.where(item_id == id_)[0][0]
        scores[pos] = min_score
    return scores

In [4]:
%%time
M = item_matrix @ item_matrix.T

CPU times: user 3.37 s, sys: 1.97 s, total: 5.33 s
Wall time: 860 ms


In [1]:
import telebot
from telebot import types

ratings = {}
request_user = {}
API_TOKEN = '5677096457:AAHYpRxfaCw9WnJjRybbF-igrxK3XAe3cXo'
bot = telebot.TeleBot(API_TOKEN)

test_ids = [20, 21, 269, 136]


@bot.message_handler(commands=['suggest'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        bot.send_message(message.chat.id,
                             text=f'Not enough info')
    else:
        p = np.zeros(13686)
        #print(ratings[name])
        for anime_id, score in ratings[name].items():
            p[anime_id] = score
        recs = M @ p
        recs = downvote_seen_items(recs, test_ids)
        for id_ in topn_recommendations(recs):
            bot.send_message(message.chat.id,
                             text=f'Consider {id2name[str(item_id[id_])]}')
        
    
                
@bot.message_handler(commands=['rate'])
def echo_message(message):
    name = message.from_user.username
    if name not in ratings:
        ratings[name] = {}
    #anime_id = test_ids[len(ratings[name])]
    #anime_id = np.where(item_id == anime_id)[0][0]
    anime_id = np.random.choice(item_id)
    request_user[name] = anime_id
    bot.send_message(message.chat.id,
                         text=f'Please rate {id2name[str(item_id[anime_id])]}')
    bot.send_message(message.chat.id,
                         text=f'You have to rate {len(test_ids) - len(ratings[name])} more')
        
        

    
@bot.message_handler(content_types=['text'])
def echo_message(message):
    name = message.from_user.username 
    if name in request_user:
        ratings[name][request_user[name]] = float(message.text)      
                
    

In [ ]:
bot.polling()